In [1]:
import numpy as np
from Assignments.Third.Model import MyFFLM, cross_entropy
import re
from tqdm import tqdm

In [2]:
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

with open("lotrFotr.txt","r",encoding='utf-8') as file:
    raw_txt = file.read()

stripped_txt_lotr = raw_txt.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')\
    .replace(';', '').replace(':', '').replace('  ', ' ').replace('.', '').replace(',', '').replace('"', '').lower()
stripped_txt_lotr = re.sub(r'[^\w\s]', '', stripped_txt_lotr).split()

# with open("mobydick.txt","r",encoding='utf-8') as file:
#     raw_txt = file.read()
#
# stripped_txt_mb = raw_txt.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')\
#     .replace(';', '').replace(':', '').replace('  ', ' ').replace('.', '').replace(',', '').lower().split()

In [3]:
vocab = list(set(stripped_txt_lotr))
vocab.sort()
vocab_dict = {}
for i, word in enumerate(vocab):
    identity_vector = np.zeros(len(vocab))
    identity_vector[i] = 1
    vocab_dict[word] = identity_vector
print(len(vocab))

9794


In [4]:
# Need to split up bc of ram issues, batches is the answer I think
memory_depth = 3
dataX = np.zeros((len(stripped_txt_lotr[:1000]) - memory_depth, memory_depth, len(vocab_dict)))
dataY = np.zeros(len(stripped_txt_lotr[:1000]) - memory_depth, dtype='int32')
for i in range(len(stripped_txt_lotr[:1000]) - memory_depth):
    x = np.zeros((memory_depth, len(vocab_dict)))
    for j in range(memory_depth):
        x[j] = vocab_dict[stripped_txt_lotr[:1000][i+j]]
    dataX[i] = x
    dataY[i] = np.argmax(vocab_dict[stripped_txt_lotr[:1000][i+memory_depth]])

In [5]:
reversed_dict = {}
for key in vocab_dict:
    reversed_dict[np.argmax(vocab_dict[key])] = key
for x in dataX[0]:
    print(reversed_dict[np.argmax(x)])
    print(x)
print(reversed_dict[dataY[0]])
print(dataY[0])

three
[0. 0. 0. ... 0. 0. 0.]
rings
[0. 0. 0. ... 0. 0. 0.]
for
[0. 0. 0. ... 0. 0. 0.]
the
8463


In [6]:
v_len = len(vocab)
data_set_length = len(stripped_txt_lotr) - memory_depth
batch_size = 60000
model = MyFFLM(v_len, 64, learning_rate=0.001, memory_depth=memory_depth)
# loss = 0
for epoch in range(10):
    for batch in range(data_set_length // batch_size):
        dataX = np.zeros((batch_size - memory_depth, memory_depth, v_len), dtype='float16')
        dataY = np.zeros((batch_size - memory_depth, v_len), dtype='int16')
        for i in range(batch_size - memory_depth):
            x = np.zeros((memory_depth, len(vocab_dict)))
            for j in range(memory_depth):
                x[j] = vocab_dict[stripped_txt_lotr[(batch*batch_size):(batch*batch_size)+batch_size][i+j]]
            dataX[i] = x
            dataY[i] = vocab_dict[stripped_txt_lotr[(batch*batch_size):(batch*batch_size)+batch_size][i+memory_depth]]
        # if epoch == 0:
        #     for words, target in zip(dataX[995:], dataY[995:]):
        #         print([reversed_dict[np.argmax(word)] for word in words], reversed_dict[np.argmax(target)])
        batch_progress = tqdm(total=dataX.shape[0], desc="batch progression {}/{}".format(batch, data_set_length // batch_size))
        for k, (x, y) in enumerate(zip(dataX, dataY)):
            y_pred = model.forward(x)
            model.backprop(y, y_pred)
            batch_progress.update()
        batch_progress.close()
            # if k % (batch_size - memory_depth - 1) == 1 and  k != 1:
            #     print("epoch:", epoch, "batch:", f"{batch}/{data_set_length//batch_size}")
            #     l = cross_entropy(y, y_pred)
            #     print("loss:", l, l[0, np.argmax(l)])
            #     print("Predicted vs actual:", reversed_dict[np.argmax(y_pred)], reversed_dict[np.argmax(y)])
            #     print("Predicted value:", y_pred[0, np.argmax(y_pred)])
        print(batch)


batch progression 0/3:   3%|▎         | 1575/59997 [00:51<32:22, 30.08it/s]

KeyboardInterrupt: 